In [3]:
import pandas as pd
import numpy as np

train_data_part1 = pd.read_csv('C:/Users/path/NumNetworks - FashionMNIST/Data/mnist_train/fashion-mnist_train1.csv')
train_data_part2 = pd.read_csv('C:/Users/path/NumNetworks - FashionMNIST/Data/mnist_train/fashion-mnist_train2.csv')

train_data = pd.concat([train_data_part1, train_data_part2])

test_data = pd.read_csv('C:/Users/path/NumNetworks - FashionMNIST/Data/mnist_test/fashion-mnist_test.csv')
train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_data = np.array(train_data)
m, n = train_data.shape
print(train_data.shape, n, m )
np.random.shuffle(train_data)

test_data = np.array(test_data)
m2, n2 =  test_data.shape
print(test_data.shape, n, m )
np.random.shuffle(test_data)

data_train = train_data.T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
print(data_train.shape, n, m )
print(Y_train.shape, X_train.shape)

data_test = test_data.T
Y_test = data_test[0]
X_test = data_test[1:n2]
X_test = X_test / 255.
print(data_test.shape, n2, m2 )
print(Y_test.shape, X_test.shape)

(60000, 785) 785 60000
(10000, 785) 785 60000
(785, 60000) 785 60000
(60000,) (784, 60000)
(785, 10000) 785 10000
(10000,) (784, 10000)


In [5]:
def initialize_parameters():
    W1 = np.random.rand(64, 784) - 0.5
    b1 = np.random.rand(64, 1) - 0.5
    W2 = np.random.rand(32, 64) - 0.5
    b2 = np.random.rand(32, 1) - 0.5
    W3 = np.random.rand(10, 32) - 0.5
    b3 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2, W3, b3

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    Z_shift = Z - np.max(Z, axis=0)
    exp_Z = np.exp(Z_shift)
    A = exp_Z / np.sum(exp_Z, axis=0)
    return A

def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3, axis=1, keepdims=True)
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2, dW3, db3

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    W3 -= alpha * dW3
    b3 -= alpha * db3
    return W1, b1, W2, b2, W3, b3

In [6]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = initialize_parameters()
    
    for i in range(1, iterations+1):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        
        if i % 10 == 0:
            print("Iteration:", i)
            predictions = get_predictions(A3)
            print("Accuracy:", get_accuracy(predictions, Y))
    
    return W1, b1, W2, b2, W3, b3

In [7]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, 0.25, 1000)

Iteration: 10
[8 8 7 ... 5 1 2] [9 9 5 ... 8 1 2]
Accuracy: 0.41146666666666665
Iteration: 20
[9 9 9 ... 6 1 2] [9 9 5 ... 8 1 2]
Accuracy: 0.5974666666666667
Iteration: 30
[9 9 5 ... 6 1 2] [9 9 5 ... 8 1 2]
Accuracy: 0.6378666666666667
Iteration: 40
[9 9 5 ... 6 1 4] [9 9 5 ... 8 1 2]
Accuracy: 0.6560833333333334
Iteration: 50
[9 9 5 ... 6 1 4] [9 9 5 ... 8 1 2]
Accuracy: 0.6710666666666667
Iteration: 60
[9 9 5 ... 6 1 4] [9 9 5 ... 8 1 2]
Accuracy: 0.6845666666666667
Iteration: 70
[9 9 5 ... 6 1 4] [9 9 5 ... 8 1 2]
Accuracy: 0.6939166666666666
Iteration: 80
[9 9 5 ... 8 1 4] [9 9 5 ... 8 1 2]
Accuracy: 0.7026833333333333
Iteration: 90
[9 9 5 ... 8 1 2] [9 9 5 ... 8 1 2]
Accuracy: 0.7093333333333334
Iteration: 100
[9 9 5 ... 8 1 2] [9 9 5 ... 8 1 2]
Accuracy: 0.7167166666666667
Iteration: 110
[9 9 5 ... 8 1 6] [9 9 5 ... 8 1 2]
Accuracy: 0.7219833333333333
Iteration: 120
[9 9 5 ... 8 1 6] [9 9 5 ... 8 1 2]
Accuracy: 0.7270833333333333
Iteration: 130
[9 9 5 ... 8 1 6] [9 9 5 ... 8 1 

In [16]:
def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
    predictions = get_predictions(A3)
    return predictions

In [19]:
test_prediction = make_predictions(X_test, W1, b1, W2, b2, W3, b3)
get_accuracy(test_prediction, Y_test)

[3 5 3 ... 2 9 5] [3 5 3 ... 6 9 5]


0.8363